![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/training/english/dl-ner/ner_logs_GCP.ipynb)


# Exporting Logs in GCP with NER training

In Spark NLP you can configure the location to download the logs of training NER models. Starting at Spark NLP 5.1.0, you can set a GCP Storage URI, or Azure Storage URI or DBFS paths like HDFS or Databricks FS.

In this notebook, we are going to see the steps required to use an external GCP Storage URI to store the logs of traning an NER model

To do this, we need to configure the spark session with the required settings for Spark NLP and Spark ML.

### Spark NLP Settings

`project_id`: We need to know the ProjectId of our GCP Storage. This is defined in `spark.jsl.settings.gcp.project_id`

To integrage with GCP, we need to setup Application Default Credentials (ADC) for GCP. You can check how to configure it in the official [GCP documentation](https://cloud.google.com/docs/authentication/provide-credentials-adc)

In [ ]:
! gcloud auth application-default login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=tFHFF9TX2zTmdvgrv8nJCMR5hzJSpO&prompt=consent&access_type=offline&code_challenge=thIesdTdE1baCvLRj3XVbC7hxiMAZpUqFh4gGgFVwpw&code_challenge_method=S256

Enter authorization code: 4/0AbUR2VNBbJPmIJ4S1SGDL0fcasgJGz2NBdJczgZhSjdYAipIjBskSuZnBTFt7oX8aD3r1w

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Cannot find a quota project to add to AD

In [ ]:
! ls /content/.config/application_default_credentials.json

/content/.config/application_default_credentials.json


### Spark ML Settings

Spark ML requires the following configuration to load a model from GCP using ADC:

1. GCP connector: You need to identify your hadoop version and set the required dependency in `spark.jars.packages`
2. ADC credentials: After following the instructions to setup ADC, you will have a JSON file that holds your authenticiation information. This file is setup in `spark.hadoop.google.cloud.auth.service.account.json.keyfile`
3. Hadoop File System: You also need to setup the Hadoop implementation to work with GCP Storage as file system. This is define in `spark.hadoop.fs.gs.impl`
3. Finally, to mitigate conflicts between Spark's dependencies and user dependencies. You must define `spark.driver.userClassPathFirst` as true. You may also need to define `spark.executor.userClassPathFirst` as true.

In [ ]:
# Only run this Cell when you are using Spark NLP on Google Colab
!wget https://setup.johnsnowlabs.com/colab.sh -O - | bash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.7/486.7 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 kB 20.0 MB/s eta 0:00:00


Now, let's take a look at a simple example the spark session creation below to see how to define each of the configurations with its values:

In [ ]:
print("Enter your GCP ProjectId:")
PROJECT_ID = input()

In [ ]:
import sparknlp
import pyspark

json_keyfile = "/content/.config/application_default_credentials.json"

#GCP Storage configuration
gcp_params = {
    "spark.jars.packages": "com.google.cloud.bigdataoss:gcs-connector:hadoop3-2.2.8",
    "spark.hadoop.fs.gs.impl": "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem",
    "spark.driver.userClassPathFirst": "true",
    "spark.hadoop.google.cloud.auth.service.account.json.keyfile": json_keyfile,
    "spark.jsl.settings.gcp.project_id": PROJECT_ID
}

spark = sparknlp.start(params=gcp_params)

print("Apache Spark version: {}".format(spark.version))

In [ ]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.training import CoNLL

In [ ]:
training_data = CoNLL().readDataset(spark, './test_ner_dataset.txt')
training_data.show(3)

embeddings = WordEmbeddingsModel.pretrained("glove_100d")
ready_data = embeddings.transform(training_data).cache()

ner_tagger = NerDLApproach() \
    .setInputCols("sentence", "token", "embeddings") \
    .setLabelColumn("label") \
    .setOutputCol("ner") \
    .setMaxEpochs(1) \
    .setMaxEpochs(5) \
    .setRandomSeed(0) \
    .setVerbose(2) \
    .setDropout(0.8) \
    .setBatchSize(18) \
    .setEnableOutputLogs(True) \
    .setOutputLogsPath("gs://test-bucket-danilo/logs")

ner_model = ner_tagger.fit(ready_data)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|John Smith works ...|[{document, 0, 35...|[{document, 0, 35...|[{token, 0, 3, Jo...|[{pos, 0, 3, NNP,...|[{named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]
